In [20]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-09-27 07:40:27.431854: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-27 07:40:27.506388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 07:40:27.537335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 07:40:27.546747: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 07:40:27.610292: I tensorflow/core/platform/cpu_feature_guar

In [21]:
# loading the data from text file 
text = open('/teamspace/studios/this_studio/Shakspear/alllines.txt', 'r').read()

In [22]:
def prepare_sequences(text, sequence_length=10):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text])
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in text.split('\n'):
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i + 1]
            input_sequences.append(n_gram_sequence)

    # Pad sequences
    input_sequences = pad_sequences(input_sequences, maxlen=sequence_length, padding='pre')
    X, y = input_sequences[:, :-1], input_sequences[:, -1]
    y = keras.utils.to_categorical(y, num_classes=total_words)

    return X, y, total_words, tokenizer

In [23]:
# Step 2: Build and Train the LSTM Model
def build_model(sequence_length, total_words):
    model = keras.Sequential()
    model.add(layers.Embedding(total_words, 100, input_length=sequence_length - 1))
    model.add(layers.LSTM(150))
    model.add(layers.Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
def train_model(X, y, model):
    model.fit(X, y, epochs=100, verbose=1)

In [25]:
def generate_next_word(model, tokenizer, input_text, sequence_length):
    input_seq = tokenizer.texts_to_sequences([input_text])[0]
    input_seq = pad_sequences([input_seq], maxlen=sequence_length - 1, padding='pre')
    predicted = model.predict(input_seq, verbose=0)
    return tokenizer.index_word[np.argmax(predicted)]


In [26]:
X, y, total_words, tokenizer = prepare_sequences(text)

model = build_model(X.shape[1], total_words)
train_model(X, y, model)

# while True:
#     input_text = input("Enter a partial sentence (or 'exit' to quit): ")
#     if input_text.lower() == 'exit':
#         break
#     next_word = generate_next_word(model, tokenizer, input_text, X.shape[1])
#     print(f"Suggested next word: {next_word}")

MemoryError: Unable to allocate 135. GiB for an array with shape (709820, 25576) and data type float64